In [1]:
from hstrader import HsTrader
from hstrader.models import Tick, Event
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta
import os
from dotenv import load_dotenv
import logging
import asyncio
import numpy as np
from plotly._subplots import make_subplots
from IPython.display import display, clear_output
import nest_asyncio

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Get the CLIENT_ID from the environment variables
id = os.getenv('CLIENT_ID')

# Get the CLIENT_SECRET from the environment variables
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

In [3]:
def wilder_smoothing(series, window=14):
    """
    Apply Wilder's smoothing technique to a sequence of numeric data.
    
    Parameters
    ---------
        series (pandas.Series): A sequence of numeric data to be smoothed.
        window (int): The period over which smoothing is applied. Default is 14.
    
    Returns
    -------
        pandas.Series: The smoothed data series.
    """
    smoothed = series.copy()
    smoothed.iloc[:window] = series.iloc[:window].mean()  # Initialize with simple average for the first value
    for i in range(window, len(series)):
        smoothed.iloc[i] = (smoothed.iloc[i - 1] * (window - 1) + series.iloc[i]) / window
    return smoothed

In [4]:
def calculate_adx(df, window=14):
    """
    Calculate the Average Directional Index (ADX), along with +DI and -DI indicators.
    
    Parameters
    ----------
        df (pandas.DataFrame): DataFrame containing 'high', 'low', and 'close' columns.
        window (int): Number of periods to use for the calculations. Default is 14.
    
    Returns
    -------
        DataFrame 
    """
    # Calculate TR, +DM, -DM
    df['prev_Close'] = df['close'].shift(1)
    df['TR'] = df[['high', 'low', 'close', 'prev_Close']].apply(lambda x: max(x['high'] - x['low'], abs(x['high'] - x['prev_Close']), abs(x['low'] - x['prev_Close'])), axis=1)
    df['+DM'] = np.where((df['high'] - df['high'].shift(1)) > (df['low'].shift(1) - df['low']), df['high'] - df['high'].shift(1), 0)
    df['-DM'] = np.where((df['low'].shift(1) - df['low']) > (df['high'] - df['high'].shift(1)), df['low'].shift(1) - df['low'], 0)
    
    # Smooth TR, +DM, -DM
    df['ATR'] = wilder_smoothing(df['TR'], window=window)
    df['+DM_smooth'] = wilder_smoothing(df['+DM'], window=window)
    df['-DM_smooth'] = wilder_smoothing(df['-DM'], window=window)
    
    # Calculate +DI, -DI
    df['+DI'] = (df['+DM_smooth'] / df['ATR']) * 100
    df['-DI'] = (df['-DM_smooth'] / df['ATR']) * 100
    
    # Calculate DX (Directional Movement Index)
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    
    # Calculate ADX (Average Directional Movement Index) by smoothing DX
    df['ADX'] = df['DX'].rolling(window=window).mean()

    return df


### Plot Candlestick Chart with Average Directional Index Indicator (Historical Market Data):

In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol=client.get_symbol('EURUSD')

 # Create a DataFrame from the retrieved data
market_history=client.get_market_history(symbol.id)
df= pd.DataFrame([bar.model_dump() for bar in market_history])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)


# Calculate ADX by calling the earlier function
df = calculate_adx(df)

# Drop rows with NaN values in 'ADX', '+DI', and '-DI'
df = df.dropna(subset=['ADX', '+DI', '-DI'])

df.head()

In [ ]:
# Create subplot with 2 rows and shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Add candlestick chart
fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick'
    )
        ,row=1, col=1)


# ADX, +DI, and -DI line charts
fig.add_trace(go.Scatter(x=df.index, y=df['ADX'], mode='lines', name='ADX', line=dict(color='blue')),
              row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['+DI'], mode='lines', name='+DI', line=dict(color='green')),
              row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['-DI'], mode='lines', name='-DI', line=dict(color='red')),
              row=2, col=1)

# Update layout
fig.update_layout(yaxis1_title='Price ', yaxis2_title='+DI / -DI / ADX',
                  showlegend=True, height=600, xaxis_rangeslider_visible=False)

# Remove x-axis tick labels for the first chart
fig.update_xaxes(showticklabels=False, row=1, col=1)

fig.show()

### Example Plot

Below is a static image of Average Directional Index (ADX) applied to historical data:
![Fibonacci Plot](img/average_directional_index.png)

### Plot Real-Time Candlestick Chart with Average Directional Index  Indicator :
If you’re interested in backtesting and strategy optimization, the code above utilizes the Average Directional Index Indicator with historical data. However, if you want real-time data for live trading decisions, you can run the code below to incorporate the indicator in live market conditions.

In [7]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol=client.get_symbol('Bitcoin')

 # Create a DataFrame from the retrieved data
market_history=client.get_market_history(symbol.id)
df= pd.DataFrame([bar.model_dump() for bar in market_history])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)


# Calculate ADX by calling the earlier function
df = calculate_adx(df)

# Drop rows with NaN values in 'ADX', '+DI', and '-DI'
df = df.dropna(subset=['ADX', '+DI', '-DI'])


In [8]:
# Setup the initial plot
fig = go.FigureWidget(make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02))

# Add candlestick chart
fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlestick'), row=1, col=1)

# ADX, +DI, and -DI line charts
fig.add_trace(go.Scatter(x=df.index, y=df['ADX'], mode='lines', name='ADX', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['+DI'], mode='lines', name='+DI', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['-DI'], mode='lines', name='-DI', line=dict(color='red')), row=2, col=1)

# Update layout
fig.update_layout(yaxis1_title='Price ', yaxis2_title='+DI / -DI / ADX', showlegend=True, height=600,xaxis_rangeslider_visible=False)

# Remove x-axis tick labels for the first charts
fig.update_xaxes(showticklabels=False, row=1, col=1)


pass

In [9]:
# Prepare data dictionary
CANDLE_INTERVAL = timedelta(minutes=1)
data = {
    'x': list(df.index),
    'open': list(df['open']),
    'high': list(df['high']),
    'low': list(df['low']),
    'close': list(df['close'])
}

In [10]:
@client.subscribe(Event.MARKET)
async def on_market(tick: Tick):
    global data, df

    try:
        if tick.symbol_id == symbol.id:
            tick_time = pd.to_datetime(tick.time)
            if tick_time >= data['x'][-1] + CANDLE_INTERVAL:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
                
            else:
                data['low'][-1] = min(tick.bid, data['low'][-1])
                data['high'][-1] = max(tick.bid, data['high'][-1])
                data['close'][-1] = tick.bid
                

            df = pd.DataFrame({
                'time': data['x'],
                'open': data['open'],
                'high': data['high'],
                'low': data['low'],
                'close': data['close']
            }).set_index('time')

            # Recalculate the Average Directional Index  componint
            df = calculate_adx(df)

            # Drop rows with NaN values in 'ADX', '+DI', and '-DI'
            df = df.dropna(subset=['ADX', '+DI', '-DI'])
    except Exception as e:
        logging.error(f"Error in on_market: {e}")
        await client.refresh_token()
    

In [11]:
async def update_plot():
    global data, df
    while True:
        await asyncio.sleep(1)# Update every second
        clear_output(wait=True)
        
        with fig.batch_update():
            # Update candlestick chart
            fig.data[0].x = data['x']
            fig.data[0].open = data['open']
            fig.data[0].high = data['high']
            fig.data[0].low = data['low']
            fig.data[0].close = data['close']
            
            # Update ADX, +DI, -DI lines
            fig.data[1].x = df.index
            fig.data[1].y = df['ADX']
            fig.data[2].x = df.index
            fig.data[2].y = df['+DI']
            fig.data[3].x = df.index
            fig.data[3].y = df['-DI']
            
        display(fig)

In [ ]:
nest_asyncio.apply()
loop=asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(client.start_async(), update_plot()))